원문: Moler, Cleve B. Numerical computing with MATLAB. Society for Industrial and Applied Mathematics, 2008.

# 1.4 마방진

마방진은 흥미로운 예제 행렬이다.

In [4]:
# scipython.com/book/chapter-6-numpy/examples/creating-a-magic-square/
import numpy as np


def magic(N):
    magic_square = np.zeros((N, N), dtype=int)
    
    n = 1
    i, j = 0, N//2
    
    while n <= N ** 2:
        magic_square[i, j] = n
        n += 1
        newi, newj = (i - 1) % N, (j + 1) % N
        if magic_square[newi, newj]:
            i += 1
        else:
            i, j = newi, newj
    
    return magic_square

A = magic(3)
A

matrix([[8, 1, 6],
        [3, 5, 7],
        [4, 9, 2]])

In [3]:
np.sum(A)

45

아래 명령은 각 

In [5]:
np.sum(A, 0)

matrix([[15, 15, 15]])

각 행의 합은 다음과 같이 구한다.

In [7]:
np.sum(A, 1)

matrix([[15],
        [15],
        [15]])

In [9]:
np.sum(np.diag(A))

15